In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('/data', one_hot=True)

Extracting /data\train-images-idx3-ubyte.gz
Extracting /data\train-labels-idx1-ubyte.gz
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz


In [31]:
def encode(lstm_enc, state, input, reuse=True):
    with tf.variable_scope("encoder", reuse=reuse):
        return lstm_enc(input, state)

def decode(lstm_dec, state, input, reuse=True):
    with tf.variable_scope("decoder", reuse=reuse):
        return lstm_dec(input, state)

def read_no_attn(x, x_hat, h_dec_prev):
    return tf.concat([x, x_hat], 1)


In [32]:
# input size
X_size = mnist.train.images.shape[1]

# latent dimension
z = 100
# sequence length
T = 10

# total time-step
glimpses = 64

enc_size = 256
dec_size = 256

# read and write size
read  = (2,2)
write = (5,5)

#
batch_size = 32

graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(tf.float32, (batch_size, X_size), 'inputs')
    
    noise = tf.random_normal((batch_size, z), dtype=tf.float32)
    
    lstm_encoder = LSTMCell(enc_size)
    encoder = lambda st, inp, reuse: encode(lstm_encoder, st, inp, reuse)
    enc_state = lstm_encoder.zero_state(batch_size, tf.float32)
    
    lstm_decoder = LSTMCell(dec_size)
    decoder = lambda st, inp, reuse: decode(lstm_decoder, st, inp, reuse)
    dec_state = lstm_decoder.zero_state(batch_size, tf.float32)

    with tf.variable_scope('l') as scope:
        for t in range(T):
            pass